# import library

In [1]:
# import all library
import librosa, IPython, datetime, time, os, sys, copy, dill, mir_eval, glob
#import pickle
import _pickle as pickle
import numpy as np
#import pandas as pd
#from scipy.spatial.distance import euclidean, pdist, squareform
#import IPython.display as ipd
#from datetime import datetime
from time import gmtime, strftime
#from imageio import imread as imread
#from imageio import imsave as imsave
#import librosa.display
import tensorflow as tf
#from pypianoroll import Multitrack, Track
from matplotlib import pyplot as plt
#from midiutil.MidiFile import MIDIFile
#import ray
from ops import *
#import pydub
#from ADTLib import ADT
import matplotlib.pyplot as plt
%matplotlib inline

# show version info
print ("[info] Current Time:     " + datetime.datetime.now().strftime('%Y/%m/%d  %H:%M:%S'))
print ("[info] Python Version:   " + sys.version.split('\n')[0].split(' ')[0])
print ("[info] Working Dir:      " + os.getcwd()+'/')

# enable gpu usage constraint here
limited_gpu_usage = 1     # keep "1" if wan to use certain GPU device
occupied_gpu_dev = 0      # 0 for GPU 0(1080Ti), 1 for GPU 1(1080Ti), 2 for GPU 2(2080Ti)

# if gpu usage is constraint, limit certain gpu for use
if (limited_gpu_usage == 1):
    # set available GPU
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"                       # list GPU sequence by PCI bus GPU ID
    #os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2"                       
    os.environ["CUDA_VISIBLE_DEVICES"] = "{}".format(occupied_gpu_dev)   

    # check available GPU
    from tensorflow.python.client import device_lib
    for x in range(1, len(device_lib.list_local_devices())):
        print ("[info] GPU " + device_lib.list_local_devices()[x].physical_device_desc)

[info] Current Time:     2019/03/05  11:55:32
[info] Python Version:   3.6.5
[info] Working Dir:      /host/home/python/musegan_npz/
[info] GPU device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:05:00.0, compute capability: 6.1


# Read train s100 index code

In [2]:
# read train s100 data
with open('./large_dataset_data/cqt_obj_pkl/song_index_s100_train_list.pkl', 'rb') as pkl_file:        
    song_index_s100_train_list_reload = pickle.load(pkl_file)
    
print ('[info] train s100 pkg is loaded.')

s100_song_index_list_train =     song_index_s100_train_list_reload[0]
s100_song_bar_num_list_train =   song_index_s100_train_list_reload[1]
s100_index_code_list_train =     song_index_s100_train_list_reload[2]

print (len(s100_song_index_list_train))
print (s100_song_index_list_train[:5])
print (len(s100_song_bar_num_list_train))
print (np.sum(s100_song_bar_num_list_train))
print (len(s100_index_code_list_train))

[info] train s100 pkg is loaded.
100
['00063', '00516', '00550', '00607', '00626']
100
11303
11303


# Read test s100 index code

In [3]:
# read test s100 data
with open('./large_dataset_data/cqt_obj_pkl/song_index_s100_test_list.pkl', 'rb') as pkl_file:        
    song_index_s100_test_list_reload = pickle.load(pkl_file)
    
print ('[info] test s100 pkg is loaded.')

s100_song_index_list_test =     song_index_s100_test_list_reload[0]
s100_song_bar_num_list_test =   song_index_s100_test_list_reload[1]
s100_index_code_list_test =     song_index_s100_test_list_reload[2]

print (len(s100_song_index_list_test))
print (s100_song_index_list_test[:5])
print (len(s100_song_bar_num_list_test))
print (np.sum(s100_song_bar_num_list_test))
print (len(s100_index_code_list_test))

[info] test s100 pkg is loaded.
100
['00244', '00353', '00367', '00395', '00680']
100
11338
11338


# show [train/test] index code

In [4]:
for x in s100_index_code_list_train[:3]:
    print (x)
print ('...')
for x in s100_index_code_list_test[:3]:
    print (x)

00063000sp0
00063001sp0
00063002sp0
...
00244000sp0
00244001sp0
00244002sp0


# check style tag data

In [5]:
midi_track_obj_folder = './large_dataset_data/midi_track_obj_each_songs/*.pkl'

midi_track_obj_folder_flist = np.sort(glob.glob(midi_track_obj_folder, recursive=True)).tolist()

print ('Midi obj files: {}'.format(len(midi_track_obj_folder_flist)))

total_midi_obj_files = len(midi_track_obj_folder_flist)

for x in midi_track_obj_folder_flist[:3]:
    print (x)
    

class midi_track(object):
    def __init__(self):
        self.file_name = ""
        self.file_msd_idx = ""
        self.file_msd_id = ""
        self.file_clnsd_id = ""
        self.mtrack_data = []
        self.mtrack4_data = []
        self.pmidi_data = []
        self.pmidi4_data_sp0 = []
        self.pmidi4_data_sp1 = []
        self.pmidi4_data_sp2 = []
        self.pmidi4_data_sp3 = []
        self.pmidi4_data_sn1 = []
        self.pmidi4_data_sn2 = []
        self.pmidi4_data_sn3 = []
        self.tempo = 0        
        self.style_tag_list = copy.deepcopy(file_unique_genre)
        self.style_tag = ""
        self.style_tag_id = 0
        self.downbeats_list_fixed = []
        self.bar_range_list_fixed = []
        self.drum_bar_list = []
        self.drum_bar_list_bin = []
        self.drum_bar_note_num = []
        
print ('MIDI track object is defined.')

def get_midi_object(obj_idx_in, obj_flist=midi_track_obj_folder_flist):
    
    file_name = obj_flist[obj_idx_in]
    
    with open(file_name, 'rb') as pkl_file:
        
        read_midi_obj = pickle.load(pkl_file)
        
        return (read_midi_obj)
    
print ('Reload function is defined.')

Midi obj files: 10122
./large_dataset_data/midi_track_obj_each_songs/midi_track_obj_00000.pkl
./large_dataset_data/midi_track_obj_each_songs/midi_track_obj_00001.pkl
./large_dataset_data/midi_track_obj_each_songs/midi_track_obj_00002.pkl
MIDI track object is defined.
Reload function is defined.


# show ID and genre

In [6]:
#for x_idx, k in enumerate(get_midi_object(0).style_tag_list):
#print ('ID: {:2d},  name: {}'.format(x_idx, k))

# pick a song to hack style

In [7]:
picked_song_id = 0

picked_song_idx_str_hack = s100_song_index_list_test[picked_song_id]
picked_song_idx_str_hack_list = [picked_song_idx_str_hack]
print ('picked song index: {}'.format(picked_song_idx_str_hack))


# show current style attribute
#picked_song_idx_str_full_idx_code = picked_song_idx_str_hack + '000sp0'
#style_id_oh = read_pkl_function(picked_song_idx_str_full_idx_code)[3]

#print('current style: \n{}'.format(style_id_oh))


# new hacked style
style_id_oh_hack = np.zeros(16).astype(np.float32)
style_id_oh_hack[15] = 100.0

print('\nhacked style: \n{}'.format(style_id_oh_hack))

style_id_oh_hack_list = [style_id_oh_hack]

picked song index: 00244

hacked style: 
[   0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
    0.    0.    0.  100.]


# set add note num [ 00 / 03 / 06 / 12 / 20]

In [8]:
chkpt_ver = 'v033'

bar_add_note_num = 0
#bar_add_note_num = 3
#bar_add_note_num = 6
#bar_add_note_num = 12
#bar_add_note_num = 20

rc_loss_ver_list = ['0p8', '1p0', '1p2']
rc_loss_ver = rc_loss_ver_list[2]

add_note_ver = 'p' + '{:0>2}'.format(bar_add_note_num) + 'n'

print ('CHKPT Ver:   {}'.format(chkpt_ver))
print ('Add note:    {}'.format(add_note_ver))
print ('Loss Ver:    {}'.format(rc_loss_ver))

CHKPT Ver:   v033
Add note:    p00n
Loss Ver:    1p2


# define funtion to get data by index code

In [9]:
# read relative bar data
with open('./large_dataset_data/cqt_obj_pkl/shcbl_short_drum_ssm.pkl', 'rb') as pkl_file:
    hscbl_shorted = pickle.load(pkl_file)


# define python read function
def read_pkl_function(index_code_in):
    
    # convert data into correct type
    if type(index_code_in)!=str:
        index_code_in = index_code_in.decode("utf-8")
        
    # extract information from index code
    song_idx_str = index_code_in[0:5]
    bar_idx_str = index_code_in[5:8]
    pitch_ver_str = index_code_in[8:11]
    
    
    # set parameter to get relative bars
    get_n_rtv_bars = 8 - 1
    rtv_bar_index = np.round(hscbl_shorted[int(song_idx_str)][int(bar_idx_str), 0, 0:get_n_rtv_bars]).astype(int)
    rtv_bar_ratio = np.hstack([1.0, hscbl_shorted[int(song_idx_str)][int(bar_idx_str), 1, 0:get_n_rtv_bars]]).astype(np.float32)
    
    # save bar_name into list
    cqt_data_fname_rtv_list = []
    
    #cqt_data_folder_name = './ext_dsk_nvme/large_dataset_data/cqt_obj_pkl/{}/'.format(pitch_ver_str) 
    cqt_data_folder_name = './large_dataset_data/cqt_obj_pkl/{}/'.format(pitch_ver_str) 
    cqt_data_fname = cqt_data_folder_name + index_code_in + '.pkl'
    
    cqt_data_fname_rtv_list.append(cqt_data_fname)
    
    # create read file names
    for bar_rtv_x in rtv_bar_index:        
        index_code_rtv = song_idx_str + '{:0>3}'.format(bar_rtv_x) + pitch_ver_str
        cqt_data_fname_rtv = cqt_data_folder_name + index_code_rtv + '.pkl'
        cqt_data_fname_rtv_list.append(cqt_data_fname_rtv)


    # save data into list
    cqt_data_rtv_bar_list = []
    
    # read relative files
    for cqt_data_fname_rtv in cqt_data_fname_rtv_list:        
        with open(cqt_data_fname_rtv, 'rb') as pkl_file:            
            cqt_data_rtv_bar_list.append(pickle.load(pkl_file))
            
            
    # load all attribute data
    #load_atb_fname = './ext_dsk_nvme/large_dataset_data/all_merged_data_attribute_only/'
    load_atb_fname = './large_dataset_data/all_merged_data_attribute_only/'
    load_atb_fname += 'all_merged_data_atri_{}.pkl'.format(song_idx_str)
    
    with open(load_atb_fname, 'rb') as pkl_file:
        attribute_data_list = pickle.load(pkl_file)
        

    # load drum_bar_data
    #drum_bar_file_name = './ext_dsk_nvme/large_dataset_data/drum_arrange_46_bar_unit/'
    drum_bar_file_name = './large_dataset_data/drum_arrange_46_bar_unit/'
    drum_bar_file_name += 'drum_bar_{}_{}.pkl'.format(song_idx_str, bar_idx_str)
    
    with open(drum_bar_file_name, 'rb') as pkl_file:
        drum_bar_data_reload = pickle.load(pkl_file)
        
    
    # process CQT data
    cqt_data_fname_rtv_mix = np.concatenate([cqt_data_rtv_bar_list[0][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[1][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[2][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[3][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[4][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[5][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[6][:,:,np.newaxis],
                                             cqt_data_rtv_bar_list[7][:,:,np.newaxis]], axis=-1)

    # convert data into right format
    cqt_data_fname_rtv_mix = cqt_data_fname_rtv_mix.astype(np.float32)
    

    # process attributes
    tempo_norm_v_oh =          attribute_data_list[1].astype(np.float32)
    
    
    #style_tag_array_oh =       attribute_data_list[2].astype(np.float32)
    if song_idx_str in picked_song_idx_str_hack_list:                        
        style_tag_array_oh = style_id_oh_hack_list[picked_song_idx_str_hack_list.index(song_idx_str)]
        
    else:
        style_tag_array_oh = attribute_data_list[2].astype(np.float32)
    
    
    song_progress_oh =         attribute_data_list[3][int(bar_idx_str),:].astype(np.float32)
    n_note_in_bar =  np.array([attribute_data_list[4][int(bar_idx_str)]]).astype(np.float32)
    
    
    # convert data into correct shape
    drum_bar_data_reload = drum_bar_data_reload[:,:,np.newaxis].astype(np.float32)
    
    
    # send back data
    return (cqt_data_fname_rtv_mix,  \
            rtv_bar_ratio,           \
            tempo_norm_v_oh,         \
            style_tag_array_oh,      \
            song_progress_oh,        \
            n_note_in_bar,           \
            drum_bar_data_reload)
    

    
# reading data example
index_code_for_test = s100_index_code_list_train[1000]

py_func_out_cqt_data,     \
py_func_out_cqt_ratio,    \
py_func_out_tempo_att,    \
py_func_out_style_att,    \
py_func_out_progress_att, \
py_func_out_n_note_att,   \
py_func_out_drum_arrange = read_pkl_function(index_code_for_test)

# show data format
print ('data out[0] shape: {}'.format(py_func_out_cqt_data.shape))
print ('data out[1] shape: {}'.format(py_func_out_cqt_ratio.shape))
print ('data out[2] shape: {}'.format(py_func_out_tempo_att.shape))
print ('data out[3] shape: {}'.format(py_func_out_style_att.shape))
print ('data out[4] shape: {}'.format(py_func_out_progress_att.shape))
print ('data out[5] shape: {}'.format(py_func_out_n_note_att.shape))
print ('data out[6] shape: {}'.format(py_func_out_drum_arrange.shape))

data out[0] shape: (84, 96, 8)
data out[1] shape: (8,)
data out[2] shape: (10,)
data out[3] shape: (16,)
data out[4] shape: (10,)
data out[5] shape: (1,)
data out[6] shape: (46, 16, 1)


# Define dataset.map function data shape

In [10]:
trf_out0_shape = py_func_out_cqt_data.shape
trf_out1_shape = py_func_out_cqt_ratio.shape
trf_out2_shape = py_func_out_tempo_att.shape
trf_out3_shape = py_func_out_style_att.shape
trf_out4_shape = py_func_out_progress_att.shape
trf_out5_shape = py_func_out_n_note_att.shape
trf_out6_shape = py_func_out_drum_arrange.shape

def tf_reshape_function(trf_out0, trf_out1, trf_out2, trf_out3, trf_out4, trf_out5, trf_out6):
    
    trf_out0.set_shape(trf_out0_shape)    # cqt data
    trf_out1.set_shape(trf_out1_shape)    # cqt data ratio
    trf_out2.set_shape(trf_out2_shape)    # tempo data
    trf_out3.set_shape(trf_out3_shape)    # style data
    trf_out4.set_shape(trf_out4_shape)    # song progress
    trf_out5.set_shape(trf_out5_shape)    # note number in bar
    trf_out6.set_shape(trf_out6_shape)    # drum arrange    
    
    return trf_out0, trf_out1, trf_out2, trf_out3, trf_out4, trf_out5, trf_out6

print('[info] \"dataset.map\" function is defined.')

[info] "dataset.map" function is defined.


# Define TF dataset API for train data

In [11]:
batch_size = 64 * 1

print ('[info] Total train index codes: {}'.format(len(s100_index_code_list_train)))

darr_train_dataset = tf.data.Dataset.from_tensor_slices((s100_index_code_list_train))
darr_train_dataset = darr_train_dataset.map(lambda index_code_train: tuple(tf.py_func(read_pkl_function,                    
                                                                                      [index_code_train],
                                                                                      [tf.float32, 
                                                                                       tf.float32, 
                                                                                       tf.float32, 
                                                                                       tf.float32, 
                                                                                       tf.float32, 
                                                                                       tf.float32, 
                                                                                       tf.float32])),
                                            num_parallel_calls=32)

darr_train_dataset = darr_train_dataset.map(tf_reshape_function, num_parallel_calls=32)
darr_train_dataset = darr_train_dataset.batch(batch_size=batch_size)

train_iter = darr_train_dataset.make_initializable_iterator()

# get batch data
batch_bar_cqt_data_train,         \
batch_bar_cqt_ratio_train,        \
batch_bar_tempo_data_train,       \
batch_bar_style_data_train,       \
batch_bar_progress_train,         \
batch_bar_note_num_train,         \
batch_bar_arrange_train = train_iter.get_next()

print('[info] TF train Data API is defined.')

[info] Total train index codes: 11303
[info] TF train Data API is defined.


# Define TF dataset API for test data

In [12]:
batch_size = 64 * 1

print ('[info] Total test index codes: {}'.format(len(s100_index_code_list_test)))

darr_test_dataset = tf.data.Dataset.from_tensor_slices((s100_index_code_list_test))
darr_test_dataset = darr_test_dataset.map(lambda index_code_test: tuple(tf.py_func(read_pkl_function,                    
                                                                                   [index_code_test],
                                                                                   [tf.float32, 
                                                                                    tf.float32, 
                                                                                    tf.float32, 
                                                                                    tf.float32, 
                                                                                    tf.float32, 
                                                                                    tf.float32, 
                                                                                    tf.float32])),
                                           num_parallel_calls=32)

darr_test_dataset = darr_test_dataset.map(tf_reshape_function, num_parallel_calls=16)
darr_test_dataset = darr_test_dataset.batch(batch_size=batch_size)

test_iter = darr_test_dataset.make_initializable_iterator()

# get batch data
batch_bar_cqt_data_test,         \
batch_bar_cqt_ratio_test,        \
batch_bar_tempo_data_test,       \
batch_bar_style_data_test,       \
batch_bar_progress_test,         \
batch_bar_note_num_test,         \
batch_bar_arrange_test = test_iter.get_next()

print('[info] TF test Data API is defined.')

[info] Total test index codes: 11338
[info] TF test Data API is defined.


# Define Encoder Model

In [13]:
# define leaky relu function
def lrelu(x, alpha=0.05):
    return tf.maximum(x, tf.multiply(x, alpha))

n_latent = 32

# define spectrogram encoder
def spec_encoder(enc_song_tempo,        # (batch_num, 10)
                 enc_style_id,          # (batch_num, 15)
                 enc_song_progress,     # (batch_num, 10)  
                 enc_spectrogram,       # (batch_num, 84, 96, 2)
                 reuse=False):
    
    with tf.variable_scope('spec_nn_enc', reuse=reuse):
        
        if reuse:
            tf.get_variable_scope().reuse_variables()
            
        else:
            assert tf.get_variable_scope().reuse is False    
        
        # define song_tempo input layer
        enc_song_tempo_i_layer = tf.layers.dense(inputs=enc_song_tempo,
                                                 units=64,
                                                 activation=lrelu,
                                                 name='enc_nn_at1')                                                    

        # define style_id input layer
        enc_style_id_i_layer = tf.layers.dense(inputs=enc_style_id,
                                               units=64,
                                               activation=lrelu,
                                               name='enc_nn_at2')          
        # define song_progress input layer
        enc_song_progress_i_layer = tf.layers.dense(inputs=enc_song_progress,
                                                    units=64,
                                                    activation=lrelu,
                                                    name='enc_nn_at3')
        
        # make padding Batch / Height / Width / Channel 
        enc_spectrogram_pad = tf.pad(enc_spectrogram, [[0, 0], [1, 1], [1, 1], [0, 0]], "CONSTANT")
                
        enc_conv_h1 = tf.nn.elu(instance_norm(conv2d(enc_spectrogram_pad, 
                                                     output_dim=48,
                                                     ks=[4,4],
                                                     s=[1,1], 
                                                     name='enc_conv1'), 'enc_bn1'))
    
        enc_conv_h2 = tf.nn.elu(instance_norm(conv2d(enc_conv_h1, 
                                                     output_dim=48,
                                                     ks=[4,4],
                                                     s=[2,2], 
                                                     name='enc_conv2'), 'enc_bn2'))
        
        enc_conv_h3 = tf.nn.elu(instance_norm(conv2d(enc_conv_h2,
                                                     output_dim=72,
                                                     ks=[4,4],
                                                     s=[2,2], 
                                                     name='enc_conv3'), 'enc_bn3'))
    
    
        # flatten conv output
        enc_convo_flat_out = tf.reshape(enc_conv_h3, [-1, np.prod(enc_conv_h3.get_shape()[1:])])
        
        # concat all decoder input layers
        enc_merged_layer = tf.concat([enc_song_tempo_i_layer,       \
                                      enc_style_id_i_layer,         \
                                      enc_song_progress_i_layer,    \
                                      enc_convo_flat_out],          \
                                     axis=1,                        \
                                     name='enc_nn_in_concat')
    
        
        enc_mlp_h1 = tf.layers.dense(inputs=enc_merged_layer,
                                 units=1024,
                                 activation=lrelu,
                                 name='enc_nn_mid_h1')
        
        enc_mlp_h2 = tf.layers.dense(inputs=enc_mlp_h1,
                                 units=1024,
                                 activation=lrelu,
                                 name='enc_nn_mid_h2')

        enc_mlp_h2m = lrelu(enc_mlp_h2 + enc_mlp_h1*0.2)
        
        enc_mlp_h3 = tf.layers.dense(inputs=enc_mlp_h2m,
                                     units=1024,
                                     activation=lrelu,
                                     name='enc_nn_mid_h3')
        
        enc_mlp_h3m = lrelu(enc_mlp_h3 + enc_mlp_h2m*0.2)      
        
        
        # define encoder output layer
        z_mean = tf.layers.dense(inputs=enc_mlp_h3m,
                                 units=n_latent,
                                 activation=None,
                                 name='enco_mean')
            
        z_std = tf.layers.dense(inputs=enc_mlp_h3m, 
                                units=n_latent, 
                                activation=None,
                                name='enco_std')
        
        z_epsilon = tf.random_normal(tf.stack([tf.shape(enc_mlp_h3m)[0], n_latent])) 
        
        z_latent  = z_mean + tf.multiply(z_epsilon, tf.exp(z_std * 0.5))
        
        enc_n_note_h1 = tf.layers.dense(inputs=enc_mlp_h3m, 
                                        units=512, 
                                        activation=lrelu,
                                        name='enc_nnp_h1')

        enc_n_note_h2 = tf.layers.dense(inputs=enc_n_note_h1, 
                                        units=512, 
                                        activation=lrelu,
                                        name='enc_nnp_h2')        
        
        enc_n_note_pridiction = tf.layers.dense(inputs=enc_n_note_h2, 
                                                units=1, 
                                                activation=tf.nn.sigmoid,
                                                name='enco_nnp_out')
        
        enc_n_note_pridiction_x256 = (enc_n_note_pridiction * 256) - 10
        
        return z_latent, z_mean, z_std, enc_n_note_pridiction_x256
    
print ('Encoder define done.')

Encoder define done.


# Defining decoder model

In [14]:
dec_output_size = np.prod(trf_out6_shape)   # bar_arrange, out[5] shape: (46, 16, 1)
dec_output_size_4d = [-1, trf_out6_shape[0], trf_out6_shape[1], 1]

# define leaky relu function
def lrelu(x, alpha=0.05):
    return tf.maximum(x, tf.multiply(x, alpha))

# define spectrogram encoder
def spec_decoder(dec_song_tempo,        # (batch_num, 10)
                 dec_style_id,          # (batch_num, 15)
                 dec_song_progress,     # (batch_num, 10)   
                 dec_bar_note_num,      # (batch_num, 1)
                 dec_z_sampled,         # (batch_num, 32)                 
                 reuse=False):
    
    with tf.variable_scope('spec_nn_dec', reuse=reuse):
        
        if reuse:
            tf.get_variable_scope().reuse_variables()
            
        else:
            assert tf.get_variable_scope().reuse is False          
            
        # define song_tempo input layer
        dec_song_tempo_i_layer = tf.layers.dense(inputs=dec_song_tempo,
                                                 units=64,
                                                 activation=lrelu,
                                                 name='dec_nn_at1')                                                    

        # define style_id input layer
        dec_style_id_i_layer = tf.layers.dense(inputs=dec_style_id,
                                               units=64,
                                               activation=lrelu,
                                               name='dec_nn_at2')          
        # define song_progress input layer
        dec_song_progress_i_layer = tf.layers.dense(inputs=dec_song_progress,
                                                    units=64,
                                                    activation=lrelu,
                                                    name='dec_nn_at3')
        
        # define bar_note_num input layer
        dec_bar_note_num_limited = tf.clip_by_value(dec_bar_note_num,
                                                    0.0,
                                                    200.0)
        
        dec_bar_note_num_i_layer = tf.layers.dense(inputs=dec_bar_note_num_limited,
                                                   units=64,
                                                   activation=lrelu,
                                                   name='dec_nn_at4')
        
        # define z input layer
        dec_z_i_layer = tf.layers.dense(inputs=dec_z_sampled,
                                        units=256,
                                        activation=lrelu,
                                        name='dec_nn_at5')
        
        # concat all decoder input layers
        dec_merged_layer = tf.concat([dec_song_tempo_i_layer,       \
                                      dec_style_id_i_layer,         \
                                      dec_song_progress_i_layer,    \
                                      dec_bar_note_num_i_layer,     \
                                      dec_z_i_layer],               \
                                     axis=1,                        \
                                     name='dec_nn_in_concat')
                
        dec_mlp_h1 = tf.layers.dense(inputs=dec_merged_layer,
                                     units=1024,
                                     activation=lrelu,
                                     name='dec_nn_mid_h1')                                     
        
        dec_mlp_h2 = tf.layers.dense(inputs=dec_mlp_h1,
                                     units=1024,
                                     activation=lrelu,
                                     name='dec_nn_mid_h2')   
        
        dec_mlp_h2m = lrelu(dec_mlp_h2 + dec_mlp_h1*0.2)
        
        dec_mlp_h3 = tf.layers.dense(inputs=dec_mlp_h2m,
                                     units=2048,
                                     activation=lrelu,
                                     name='dec_nn_mid_h3')
        
        dec_mlp_h4 = tf.layers.dense(inputs=dec_mlp_h3,
                                     units=2048,
                                     activation=lrelu,
                                     name='dec_nn_mid_h4')
        
        dec_mlp_h4m = lrelu(dec_mlp_h4 + dec_mlp_h3*0.2)
        
        dec_mlp_h5 = tf.layers.dense(inputs=dec_mlp_h4m,
                                     units=2048,
                                     activation=lrelu,
                                     name='dec_nn_mid_h5')
        
        dec_mlp_h5m = lrelu(dec_mlp_h5 + dec_mlp_h4m*0.2)
        
        dec_mlp_h6 = tf.layers.dense(inputs=dec_mlp_h5m,
                                     units=2048,
                                     activation=lrelu,
                                     name='dec_nn_mid_h6')
        
        dec_mlp_h6m = lrelu(dec_mlp_h6 + dec_mlp_h5m*0.2)
        
        # final output layer use tanh
        dec_mlp_output = tf.layers.dense(inputs=dec_mlp_h6m,
                                         units=dec_output_size,                        
                                         activation=tf.nn.tanh,
                                         name='dec_nn_out_final')        
        
        # normalize output range to -1.5 ~ 2.5
        #dec_mlp_output_norm = (dec_mlp_output * 4.0) + 0.5        
        dec_mlp_output_norm = lrelu((dec_mlp_output * 2.0) + 0.5)
        
        # reshape data into 4d shape
        dec_output_reshape = tf.reshape(dec_mlp_output_norm, dec_output_size_4d)
        
        return dec_output_reshape

print ('Decoder define done.')

Decoder define done.


# define function to apply cqt ratio

In [15]:
def tf_broadcast(tensor, shape):
    
    return (tensor + tf.zeros(shape, dtype=tensor.dtype))

def apply_cqt_ratio(cqt_tensor_in, ratio_tensor_in):
    
    ratio_tensor_in_4d = tf.reshape(ratio_tensor_in, tf.shape(cqt_tensor_in[:,0:1,0:1,:]))
    
    ratio_tensor_in_expanded = tf_broadcast(ratio_tensor_in_4d, tf.shape(cqt_tensor_in[:,:,:,:]))
    
    return (tf.multiply(cqt_tensor_in, ratio_tensor_in_expanded))

# define function to calculate diff y layer

In [16]:
# note: input tensor must be 4-D data
def get_matx_2_layer_tf(matx_data_in):
    
    matx_layer_0 = matx_data_in
    
    matx_data_pady = tf.pad(matx_data_in,
                            paddings=[[0,0], [0,0], [1,0], [0,0]],
                            mode='CONSTANT',
                            name='tf_diff2_pady',
                            constant_values=0
                            )[:, :, :-1, :]
                            #)[:, :matx_data_in.get_shape()[1], :, :]
        
    matx_layer_1 = matx_data_in - matx_data_pady
    
    matx_layer_1_concat = tf.concat([tf.zeros_like(matx_layer_1)[:,:,0:1,:], 
                                     matx_layer_1[:,:,1:,:]],
                                    axis=2)
    
    
    matx_layer_all = tf.concat([matx_layer_0, matx_layer_1_concat], 
                               axis=-1,
                               name='tf_diff2_concat')
    
    return matx_layer_all    
    
print ('Diff-2 function define done.')

Diff-2 function define done.


# define function to calculate 3 layer (add diif x/y layer)

In [17]:
# note: input tensor must be 4-D data
def get_matx_3_layer_tf(matx_data_in):
    
    matx_layer_0 = matx_data_in
    
    matx_data_pady = tf.pad(matx_data_in,
                            paddings=[[0,0], [0,0], [1,0], [0,0]],
                            mode='CONSTANT',
                            name='tf_diff3_pady',
                            constant_values=0
                            )[:, :, :-1, :]
                            #)[:, :matx_data_in.get_shape()[1], :, :]
        
    matx_layer_1 = matx_data_in - matx_data_pady
    
    matx_layer_1_concat = tf.concat([tf.zeros_like(matx_layer_1)[:,:,0:1,:], 
                                     matx_layer_1[:,:,1:,:]],
                                    axis=2)
    
    matx_data_padx = tf.pad(matx_data_in,
                            paddings=[[0,0], [1,0], [0,0], [0,0]],
                            mode='CONSTANT',
                            name='tf_diff3_padx',
                            constant_values=0
                            )[:, :-1, :, :]
                            #)[:, :, :matx_data_in.get_shape()[2], :]    
    
    matx_layer_2 = matx_data_in - matx_data_padx
    
    matx_layer_2_concat = tf.concat([tf.zeros_like(matx_layer_1)[:,0:1,:,:], 
                                     matx_layer_1[:,1:,:,:]],
                                    axis=1)
    
    matx_layer_all = tf.concat([matx_layer_0, matx_layer_1_concat, matx_layer_2_concat], 
                               axis=-1,
                               name='tf_diff3_concat')
    
    return matx_layer_all    
    
print ('Diff-3 function define done.')

Diff-3 function define done.


# Make Model Connections

In [18]:
# connect model for training data
processed_cqt_data_train = apply_cqt_ratio(batch_bar_cqt_data_train, batch_bar_cqt_ratio_train)
processed_cqt_data_double_layer_train = get_matx_2_layer_tf(processed_cqt_data_train)

vae_latent_z_train,                         \
vae_latent_zmn_train,                       \
vae_latent_zsd_train,                       \
vae_note_pred_train = spec_encoder(batch_bar_tempo_data_train,                 \
                                   batch_bar_style_data_train,                 \
                                   batch_bar_progress_train,                   \
                                   processed_cqt_data_double_layer_train,      \
                                   reuse=False)

vae_drum_out_train = spec_decoder(batch_bar_tempo_data_train,                  \
                                  batch_bar_style_data_train,                  \
                                  batch_bar_progress_train,                    \
                                  vae_note_pred_train,                         \
                                  vae_latent_z_train,                          \
                                  reuse=False)

print ('[info] VAE train model is connected.')



# connect model for testing data
processed_cqt_data_test = apply_cqt_ratio(batch_bar_cqt_data_test, batch_bar_cqt_ratio_test)
processed_cqt_data_double_layer_test = get_matx_2_layer_tf(processed_cqt_data_test)

vae_latent_z_test,     \
vae_latent_zmn_test,   \
vae_latent_zsd_test,   \
vae_note_pred_test = spec_encoder(batch_bar_tempo_data_test,                   \
                                  batch_bar_style_data_test,                   \
                                  batch_bar_progress_test,                     \
                                  processed_cqt_data_double_layer_test,        \
                                  reuse=tf.AUTO_REUSE)

vae_drum_out_test = spec_decoder(batch_bar_tempo_data_test,                    \
                                 batch_bar_style_data_test,                    \
                                 batch_bar_progress_test,                      \
                                 vae_note_pred_test + bar_add_note_num,        \
                                 vae_latent_z_test,                            \
                                 reuse=tf.AUTO_REUSE)

print ('[info] VAE test model is connected.')

[info] VAE train model is connected.
[info] VAE test model is connected.


# Define all trainable variable

In [19]:
# Define all trainable variable
t_vars = tf.trainable_variables()

# count model trainable variables
print('[info] Total params: {}'.format(np.sum([np.prod(v.shape) for v in t_vars]).value))
print('[info] Encoder params: {}'.format(np.sum([np.prod(v.shape) for v in t_vars if 'spec_nn_enc' in v.name]).value))
print('[info] Decoder params: {}\n'.format(np.sum([np.prod(v.shape) for v in t_vars if 'spec_nn_dec' in v.name]).value))

# collect all tf variables
nn_model_vars = [var for var in t_vars if 'spec_nn_enc' in var.name]
nn_model_vars.extend([var for var in t_vars if 'spec_nn_dec' in var.name])

# show all vars
print([var.name for var in nn_model_vars])

[info] Total params: 61590385
[info] Encoder params: 43808081
[info] Decoder params: 17782304

['spec_nn_enc/enc_nn_at1/kernel:0', 'spec_nn_enc/enc_nn_at1/bias:0', 'spec_nn_enc/enc_nn_at2/kernel:0', 'spec_nn_enc/enc_nn_at2/bias:0', 'spec_nn_enc/enc_nn_at3/kernel:0', 'spec_nn_enc/enc_nn_at3/bias:0', 'spec_nn_enc/enc_conv1/Conv/weights:0', 'spec_nn_enc/enc_bn1/scale:0', 'spec_nn_enc/enc_bn1/offset:0', 'spec_nn_enc/enc_conv2/Conv/weights:0', 'spec_nn_enc/enc_bn2/scale:0', 'spec_nn_enc/enc_bn2/offset:0', 'spec_nn_enc/enc_conv3/Conv/weights:0', 'spec_nn_enc/enc_bn3/scale:0', 'spec_nn_enc/enc_bn3/offset:0', 'spec_nn_enc/enc_nn_mid_h1/kernel:0', 'spec_nn_enc/enc_nn_mid_h1/bias:0', 'spec_nn_enc/enc_nn_mid_h2/kernel:0', 'spec_nn_enc/enc_nn_mid_h2/bias:0', 'spec_nn_enc/enc_nn_mid_h3/kernel:0', 'spec_nn_enc/enc_nn_mid_h3/bias:0', 'spec_nn_enc/enco_mean/kernel:0', 'spec_nn_enc/enco_mean/bias:0', 'spec_nn_enc/enco_std/kernel:0', 'spec_nn_enc/enco_std/bias:0', 'spec_nn_enc/enc_nnp_h1/kernel:0', 'spe

# Run testing loops here

In [24]:
# Testing parameters setting
chkpt_ver = chkpt_ver
batch_size = 64
show_info_epoch = 1
show_info_batch = 10
run_epoch_ratio_train = 1.0
run_epoch_ratio_test = 1.0


print ("[info] Testing job runing...")
print ('[info] ' + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S') + '\n')

# init tensorflow variables
init = tf.global_variables_initializer()
saver = tf.train.Saver(var_list=nn_model_vars)
darr_vae_config = tf.ConfigProto(allow_soft_placement=True)
darr_vae_config.gpu_options.allow_growth = True

# run TF session here
with tf.Session(config=darr_vae_config) as sess:
    
    start_time = datetime.datetime.now()
    
    sess.run(init)
    
    # reload model
    saver.restore(sess, './darr_vae_model_ld_{}/darr_vae_model_ld_{}.ckpt'.format(chkpt_ver, chkpt_ver))
    print ('[info] Model parameters are loaded.\n')
    
    epoch_target = 1
    
    # run Epoch loop here
    for epoch_idx in range(0, epoch_target):                
        
        #################################################
        #                                               #
        #    run train data session starts from here    #  
        #                                               #
        #################################################
              
        print ('[info] Test [train data] start...')
        
        sess.run(train_iter.initializer)

        note_score_train_list = []

        session_bar_cqt_data_train_list = []
        session_bar_note_num_train_list = []
        session_bar_note_num_pred_train_list = []
        session_bar_arrange_train_list = []
        session_darr_output_train_list = []
                
        data_set_len_train = len(s100_index_code_list_train)
        batch_target_train = int((data_set_len_train*run_epoch_ratio_train)//batch_size)
        #batch_target_train = 1
        
        batch_runned_train = 0 
        
        # run batch loop here
        for batch_idx in range(0, batch_target_train):            
            
            # run model testing            
            session_bar_cqt_data_train,             \
            session_bar_style_data_train,           \
            session_bar_tempo_data_train,           \
            session_bar_note_num_train,             \
            session_bar_progress_train,             \
            session_bar_arrange_train,              \
            session_bar_note_num_pred_train,        \
            session_darr_output_train = sess.run([batch_bar_cqt_data_train,             \
                                                  batch_bar_style_data_train,           \
                                                  batch_bar_tempo_data_train,           \
                                                  batch_bar_note_num_train,             \
                                                  batch_bar_progress_train,             \
                                                  batch_bar_arrange_train,              \
                                                  vae_note_pred_train,                  \
                                                  vae_drum_out_train])
            
            # get correct shape of data
            session_bar_cqt_data_train = session_bar_cqt_data_train.copy()[:,:,:,0]
            session_bar_arrange_train = session_bar_arrange_train.copy()[:,:,:,0]
            session_darr_output_train = session_darr_output_train.copy()[:,:,:,0]
            
            # calculate note score
            bin_thv = 0.5
            session_darr_output_train_bin = np.where(session_darr_output_train>=bin_thv,
                                                     np.ones_like(session_darr_output_train),
                                                     np.zeros_like(session_darr_output_train))  

            note_score_train = 1.0 - np.sum(np.abs(  \
                session_bar_arrange_train - session_darr_output_train_bin))/np.prod(session_bar_arrange_train.shape)
            
            note_score_train_list.append(note_score_train)
            
            

            # record every batch data
            if len(session_bar_cqt_data_train_list)<200:
                session_bar_cqt_data_train_list.append(session_bar_cqt_data_train)
                session_bar_note_num_train_list.append(session_bar_note_num_train)
                session_bar_note_num_pred_train_list.append(session_bar_note_num_pred_train)
                session_bar_arrange_train_list.append(session_bar_arrange_train)
                session_darr_output_train_list.append(session_darr_output_train)
            
            # record runned batch
            batch_runned_train += 1
            
            if (batch_runned_train%show_info_batch)==0:
                out_msg = "[info] Train batch done: [ {} / {} ]".format(batch_runned_train, batch_target_train)
                out_msg += ",  Note score: {:.4f} %".format(100*np.mean(note_score_train_list[-show_info_batch:]))
                print (out_msg)        

        # run train data session end from here                  
        print ('[info] Train session is finished.\n\n')
        
        
            
        ################################################
        #                                              #
        #    run test data session starts from here    #  
        #                                              #
        ################################################

        print ('[info] Test [test data] start...')
        
        sess.run(test_iter.initializer)

        note_score_test_list = []

        session_bar_cqt_data_test_list = []
        session_bar_note_num_test_list = []
        session_bar_note_num_pred_test_list = []
        session_bar_arrange_test_list = []
        session_darr_output_test_list = []
        
        session_zmn_list = []
        session_zsd_list = []
        session_zvalue_list = []
        
        data_set_len_test = len(s100_index_code_list_test)
        batch_target_test = int((data_set_len_test*run_epoch_ratio_test)//batch_size)
        
        batch_runned_test = 0 
        
        # run batch loop here
        for batch_idx in range(0, batch_target_test):            
            
            # run model testing            
            session_bar_cqt_data_test,             \
            session_bar_style_data_test,           \
            session_bar_tempo_data_test,           \
            session_bar_note_num_test,             \
            session_bar_progress_test,             \
            session_bar_arrange_test,              \
            session_bar_note_num_pred_test,        \
            session_darr_output_test,              \
            session_zmn,                           \
            session_zsd,                           \
            session_zvalue = sess.run([batch_bar_cqt_data_test,            \
                                                 batch_bar_style_data_test,          \
                                                 batch_bar_tempo_data_test,          \
                                                 batch_bar_note_num_test,            \
                                                 batch_bar_progress_test,            \
                                                 batch_bar_arrange_test,             \
                                                 vae_note_pred_test,                 \
                                                 vae_drum_out_test, vae_latent_zmn_test, vae_latent_zsd_test, vae_latent_z_test])
            
            # get correct shape of data
            session_bar_cqt_data_test = session_bar_cqt_data_test.copy()[:,:,:,0]
            session_bar_arrange_test = session_bar_arrange_test.copy()[:,:,:,0]
            session_darr_output_test = session_darr_output_test.copy()[:,:,:,0]
            
            # calculate note score
            bin_thv = 0.5
            session_darr_output_test_bin = np.where(session_darr_output_test>=bin_thv,
                                                     np.ones_like(session_darr_output_test),
                                                     np.zeros_like(session_darr_output_test))  

            note_score_test = 1.0 - np.sum(np.abs(  \
                session_bar_arrange_test - session_darr_output_test_bin))/np.prod(session_bar_arrange_test.shape)
            
            note_score_test_list.append(note_score_test)
            
            

            # record every batch data
            if len(session_bar_cqt_data_test_list)<200:
                session_bar_cqt_data_test_list.append(session_bar_cqt_data_test)
                session_bar_note_num_test_list.append(session_bar_note_num_test)
                session_bar_note_num_pred_test_list.append(session_bar_note_num_pred_test)
                session_bar_arrange_test_list.append(session_bar_arrange_test)
                session_darr_output_test_list.append(session_darr_output_test)
                
                session_zmn_list.append(session_zmn)
                session_zsd_list.append(session_zsd)
                session_zvalue_list.append(session_zvalue)
                
            
            # record runned batch
            batch_runned_test += 1
            
            if (batch_runned_test%show_info_batch)==0:
                out_msg = "[info] test batch done: [ {} / {} ]".format(batch_runned_test, batch_target_test)
                out_msg += ",  Note score: {:.4f} %".format(100*np.mean(note_score_test_list[-show_info_batch:]))
                print (out_msg)        

        # test data session end            
        print ('[info] Test session is finished.\n')
        
        
        
        ###############################
        #                             #
        #    show session info here   #  
        #                             #
        ###############################            
            
        delta_time = datetime.datetime.now() - start_time
        
        out_msg =  "\n[info] Total batch done: [ {} ]".format(batch_target_train + batch_target_test)
        out_msg += "\n[info] Train note score(Avg.): {:.4f} %".format(100*np.mean(note_score_train_list))
        out_msg += "\n[info] Train error notes per bar({} notes): {:.4f}".format(dec_output_size, 
                                                                                (1-np.mean(note_score_train_list))*dec_output_size)
        out_msg += "\n[info] Test note score(Avg.): {:.4f} %".format(100*np.mean(note_score_test_list))
        out_msg += "\n[info] test error notes per bar({} notes): {:.4f}".format(dec_output_size, 
                                                                                (1-np.mean(note_score_test_list))*dec_output_size)
        out_msg += "\n[info] Elapse Time: {}".format(str(delta_time)[:-7])
        
        print (out_msg)
        
# show process is end
print ("\n[info] Testing process is finished.")
print ("[info] " + datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

[info] Testing job runing...
[info] 2019-03-05 11:59:00

INFO:tensorflow:Restoring parameters from ./darr_vae_model_ld_v033/darr_vae_model_ld_v033.ckpt
[info] Model parameters are loaded.

[info] Test [train data] start...
[info] Train batch done: [ 10 / 176 ],  Note score: 98.8103 %
[info] Train batch done: [ 20 / 176 ],  Note score: 99.7461 %
[info] Train batch done: [ 30 / 176 ],  Note score: 99.8686 %
[info] Train batch done: [ 40 / 176 ],  Note score: 99.5372 %
[info] Train batch done: [ 50 / 176 ],  Note score: 98.7615 %
[info] Train batch done: [ 60 / 176 ],  Note score: 99.2744 %
[info] Train batch done: [ 70 / 176 ],  Note score: 99.2998 %
[info] Train batch done: [ 80 / 176 ],  Note score: 99.5514 %
[info] Train batch done: [ 90 / 176 ],  Note score: 99.9412 %
[info] Train batch done: [ 100 / 176 ],  Note score: 99.6357 %
[info] Train batch done: [ 110 / 176 ],  Note score: 99.3311 %
[info] Train batch done: [ 120 / 176 ],  Note score: 98.7831 %
[info] Train batch done: [ 130

# Check latent space z data format

In [36]:
print (len(session_zmn_list))
#print (len(session_zsd_list))
#print (len(session_zvalue_list))
#print (session_zmn_list[0].shape)
#print (session_zsd_list[0].shape)
#print (session_zvalue_list[0].shape)

session_zmn_ary =     np.concatenate(session_zmn_list, axis=0)
session_zsd_ary =     np.concatenate(session_zsd_list, axis=0)
session_zvu_ary =     np.concatenate(session_zvalue_list, axis=0)

print (session_zmn_ary.shape)
#print (session_zsd_ary.shape)
#print (session_zvu_ary.shape)

177
(11328, 32)


# save latent Z values

In [37]:
dump_file = 0

latent_z_data_pkg = [session_zmn_ary,
                     session_zsd_ary,
                     session_zvu_ary]

pkg_file_name = './large_dataset_data/model_test_result/s100_z_value_pkg.pkl'
ensure_dir(pkg_file_name)

if dump_file==1:
    with open(pkg_file_name, 'wb') as pkl_file:
        pickle.dump(latent_z_data_pkg, pkl_file)
    
    print ('[info] Z value pkl file is saved.')
    
else:
    print ('[info] No need to save file.')

[info] No need to save file.


# ensure DIR

In [29]:
def ensure_dir(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

# save s100 p00n/p03n/p06n/p12n/p20n test result here

In [22]:
test_result_train_pp = [session_bar_cqt_data_train_list,
                        session_bar_note_num_train_list,
                        session_bar_note_num_pred_train_list,
                        session_bar_arrange_train_list,
                        session_darr_output_train_list]

test_result_test_pp = [session_bar_cqt_data_test_list,
                       session_bar_note_num_test_list,
                       session_bar_note_num_pred_test_list,
                       session_bar_arrange_test_list,
                       session_darr_output_test_list]

test_s100_result_pkg = [test_result_train_pp, 
                        test_result_test_pp]


test_s100_result_fname = './large_dataset_data/model_test_result/{}/rc_loss_{}/'.format(chkpt_ver, rc_loss_ver)
test_s100_result_fname += 'test_s100_{}_result_pkg.pkl'.format(add_note_ver)

ensure_dir(test_s100_result_fname)

with open(test_s100_result_fname, 'wb') as pkl_file:
    pickle.dump(test_s100_result_pkg, pkl_file)
    
print ('[info] s100 {}, {}, {} test result saved.'.format(chkpt_ver, rc_loss_ver, add_note_ver))
print ('[info] saved file:  \"{}\"'.format(test_s100_result_fname))

[info] s100 v033, p20n test result saved.
[info] saved file:  "./large_dataset_data/model_test_result/v033/test_s100_p20n_result_pkg.pkl"


# restart kernel after run

In [ ]:
%%javascript
require(
    ["base/js/dialog"], 
    function(dialog) {
        dialog.modal({
                title: 'Notebook Halted',
                body: 'This notebook is no longer running; the kernel has been halted. Close the browser tab, or, to continue working, restart the kernel.',
                buttons: {
                    'Kernel restart': { click: function(){ Jupyter.notebook.session.restart(); } }
                }
        });
    }
);
Jupyter.notebook.session.delete();

<IPython.core.display.Javascript object>